In [ ]:
# Master BEFORE exporting 

# EFFECTS

In [27]:
## REVERB

## LOAD REVERB FUNCTIONS


# AUDIO DATA Transfer DIREC and read any WAV
direc_wav = "_pod_1_recording"
direc_wav_out = f"{direc_wav}/_edit_phase/"

#file_path = f"{direc_wav}/_0_vr_raw_.wav" # RAW
file_path = f"{direc_wav}/_0_EDIT_PHASE.wav"  # w DEMCUS
file_path = f"{direc_wav}/trial.wav"  # w TRIAL


file_out = f"{direc_wav}/_edit_phase/_0_vr"

#DEPLOY FN
############
############
edit_type = "reverb_gral"
#wav_reverb_gral(file_path, f"{file_out}_{edit_type}.wav",reverb_strength=0.01)

############ REVEBR mid- high
edit_type = "reverb_mid_high"
#wav_reverb_mod_mid_high(file_path, f"{file_out}_{edit_type}.wav", reverb_strength=1.9, split_freq=100)

############ REVEBR mid- high
edit_type = "reverb_mid_high_mod_2"
wav_reverb_mod_mid_high_mod_2(file_path, f"{file_out}_{edit_type}.wav")


In [7]:
# FN

In [7]:
import librosa
import soundfile as sf
import numpy as np

def wav_reverb_gral(filename, output_filename, reverb_strength=0.3):
    # Load the audio file
    audio, sr = librosa.load(filename, sr=None, mono=True)

    # Create a reverb kernel (this is a simplified example of a reverb effect)
    kernel_size = int(sr * 0.3)  # 0.3 seconds kernel
    kernel = np.zeros(kernel_size)
    decay = reverb_strength / kernel_size
    for i in range(kernel_size):
        kernel[i] = decay * (kernel_size - i)

    # Apply the reverb kernel
    reverbed_audio = np.convolve(audio, kernel, mode='full')

    # Trim or pad the audio to match original length
    target_length = len(audio)
    reverbed_audio = librosa.util.fix_length(reverbed_audio,size=len(audio))

    # Normalize the audio to the peak amplitude of 0.95
    max_amp = np.max(np.abs(reverbed_audio))
    if max_amp > 0:
        reverbed_audio = reverbed_audio / max_amp * 0.95

    # Save the processed audio
    sf.write(output_filename, reverbed_audio, sr)

In [18]:
import librosa
import soundfile as sf
import numpy as np

def wav_reverb_mod_mid_high(filename, output_filename, reverb_strength=0.3, split_freq=500):
    # Load the audio file
    audio, sr = librosa.load(filename, sr=None, mono=True)

    # Apply a high-pass filter to isolate mid-high frequencies
    mid_high_audio = librosa.effects.preemphasis(audio, coef=0.97, zi=[split_freq / sr])

    # Create a reverb kernel for the mid-high frequencies
    kernel_size = int(sr * 0.3)  # 0.3 seconds kernel
    kernel = np.zeros(kernel_size)
    decay = reverb_strength / kernel_size
    for i in range(kernel_size):
        kernel[i] = decay * (kernel_size - i)

    # Apply the reverb kernel
    reverbed_mid_high = np.convolve(mid_high_audio, kernel, mode='full')
    reverbed_mid_high = librosa.util.fix_length(reverbed_mid_high, size=len(audio))

    # Apply a low-pass filter to isolate low frequencies
    low_pass_filter = np.ones(len(audio)) - librosa.effects.preemphasis(audio, coef=0.97, zi=[split_freq / sr])

    # Combine the low frequencies with the reverbed mid-high frequencies
    combined_audio = (low_pass_filter * audio) + reverbed_mid_high

    # Normalize the combined audio to the peak amplitude of 0.95
    max_amp = np.max(np.abs(combined_audio))
    if max_amp > 0:
        combined_audio = combined_audio / max_amp * 0.95

    # Save the processed audio
    sf.write(output_filename, combined_audio, sr)

# Example usage


In [26]:
from pydub import AudioSegment, effects

def wav_reverb_mod_mid_high_mod_2(filename, output_filename):
    # Load the audio file
    audio = AudioSegment.from_file(filename)

    # Split the audio into frequency bands
    low_freq_band = audio.low_pass_filter(50)  # Adjust the frequency as needed
    mid_high_freq_band = audio.high_pass_filter(400)

    # Apply a complex reverb to the mid-high frequencies
    # This is a simplified example. You may need to use a dedicated DSP library or
    # DAW plugin for more sophisticated reverb effects.
    reverbed_mid_high = mid_high_freq_band.overlay(mid_high_freq_band, gain_during_overlay=-15)

    # Combine the processed and unprocessed bands
    combined_audio = low_freq_band.overlay(reverbed_mid_high)

    # Normalize the combined audio
    normalized_audio = effects.normalize(combined_audio, headroom=-0.7)  # adjust headroom as needed

    # Export the processed audio
    normalized_audio.export(output_filename, format='wav')

# Example usage
